In [352]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Menyiapkan semua library yang dibutuhkan

In [353]:
# import packages
import csv
import numpy as np
import pandas as pd
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, losses
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt




### Data Wrangling

#### Gathering Data

In [354]:
# import data
dataset1_df = pd.read_csv('dataset/dataset1.csv')
dataset1_df #show data below

,ingredient_name,rating,functions,link
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...
...,...,...,...,...
26082,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN
26083,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN
26084,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN
26085,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN


In [355]:
# import data
dataset2_df = pd.read_csv('dataset/dataset2.csv')
dataset2_df #show data below

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Evens Skin Tone, Dark Spot Fading",Antioxidant,A more stable derivative of pure vitamin C (as...,"Free Radical Biology and Medicine, September ...",INGREDIENT DICTIONARY/SEARCH RESULTS\n3-O Ethy...,"3-O ethyl ascorbic acid is a stable, water- an...","3-O ethyl ascorbic acid is a stable, water- an..."
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Soothing","Antioxidant, Plant Extracts",NaN,NaN,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcai\nRa...,Acai (pronounced “ah-sigh-ee”) is a small berr...,Acai (pronounced “ah-sigh-ee”) is a small berr...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,"Antioxidant, Plant Extracts",NaN,"Journal of AOAC International, September 2018...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcerola ...,Acerola fruit extract (also known as _Malpighi...,Acerola fruit extract (also known as _Malpighi...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...,Hydration,Antioxidant,NaN,Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl C...,NaN,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,Peptides,Lab-made anti-aging peptide that promotes firm...,"Pharmaceuticals, July 2021, pages 1–22;;Jour...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl D...,Acetyl dipeptide-1 cetyl ester is a lab-engine...,Acetyl dipeptide-1 cetyl ester is a lab-engine...
...,...,...,...,...,...,...,...,...,...,...,...
26069,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Skin Conditioning. We have not yet...
26070,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Antioxidant. We have not yet rated...
26071,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Humectant. We have not yet rated t...
26072,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Functions: Humectant, Skin Conditioning. We h..."


### Assesing Data

#### Menilai Tabel dataset1_df

In [356]:
dataset1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26087 entries, 0 to 26086
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26087 non-null  object
 1   rating           26087 non-null  object
 2   functions        26072 non-null  object
 3   link             2412 non-null   object
dtypes: object(4)
memory usage: 815.3+ KB


In [357]:
dataset1_df.isna().sum()

ingredient_name        0
rating                 0
functions             15
link               23675
dtype: int64

In [358]:
print("Jumlah Duplikasi: ", dataset1_df.duplicated().sum())

Jumlah Duplikasi:  0


In [359]:
dataset1_df.describe()

,ingredient_name,rating,functions,link
count,26087,26087,26072,2412
unique,26084,6,4055,2412
top,Rubus Idaeus (Raspberry) Fruit Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...
freq,2,23675,5642,1


#### Menilai Tabel dataset2_df

In [360]:
dataset2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26074 non-null  object
 1   rating           26074 non-null  object
 2   functions        26060 non-null  object
 3   link             2399 non-null   object
 4   benefits         1524 non-null   object
 5   categories       2350 non-null   object
 6   glance           942 non-null    object
 7   references       880 non-null    object
 8   all              2382 non-null   object
 9   description      2370 non-null   object
 10  combined_text    26073 non-null  object
dtypes: object(11)
memory usage: 2.2+ MB


In [361]:
dataset2_df.isna().sum()

ingredient_name        0
rating                 0
functions             14
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64

In [362]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


In [363]:
dataset2_df.describe()

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
count,26074,26074,26060,2399,1524,2350,942,880,2382,2370,26073
unique,26074,6,4044,2399,90,392,927,367,2382,2350,4245
top,Zymomonas Ferment Filtrate,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...,Hydration,"Antioxidant, Plant Extracts","Is a skin-restoring ingredient, meaning it is ...",Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nZingiber...,Used as an emollient and thickening agent in c...,Functions: Skin Conditioning. We have not yet...
freq,1,23675,5642,1,501,255,3,241,1,7,5642


### Cleaning Data

#### Cleaning Tabel dataset1_df

##### Duplicate Data

In [364]:
dataset1_df.duplicated().sum()

np.int64(0)

In [365]:
dataset1_df.drop_duplicates(inplace=True)

In [366]:
print("Jumlah duplikasi: ", dataset1_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [367]:
dataset1_df.isna().sum()

ingredient_name        0
rating                 0
functions             15
link               23675
dtype: int64

In [368]:
dataset1_df['functions'] = dataset1_df['functions'].fillna("Unknown")

In [369]:
dataset1_df['link'] = dataset1_df['link'].fillna("No Link")

In [370]:
print(dataset1_df.isna().sum())


ingredient_name    0
rating             0
functions          0
link               0
dtype: int64


#### Cleaning Tabel dataset2_df

##### Duplicate Data

In [371]:
dataset2_df.duplicated().sum()

np.int64(0)

In [372]:
dataset2_df.drop_duplicates(inplace=True)

In [373]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [374]:
dataset2_df.isna().sum()

ingredient_name        0
rating                 0
functions             14
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64

In [375]:
dataset2_df['functions'] = dataset2_df['functions'].fillna("Unknown")

In [376]:
print(dataset2_df.isna().sum())

ingredient_name        0
rating                 0
functions              0
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64


### Drop Column Link

In [377]:
dataset1_df = dataset1_df.drop(columns='link')

In [378]:
dataset2_df = dataset2_df.drop(columns=['benefits', 'categories', 'glance', 'references', 'all', 'description', 'combined_text', 'link'])

### Exploratory Data Analysis (EDA)

#### Explore Tabel dataset1_df

In [379]:
dataset1_df.sample(5)

,ingredient_name,rating,functions
14114,Liatris Odoratissima Leaf Oil,NOT RATED,Functions: Fragrance. We have not yet rated th...
8014,Crocus Chrysanthus Bulb Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
12087,Hydrogenated Raspberry Seed Oil,NOT RATED,Functions: Emollient. We have not yet rated th...
20906,Pyrus Calleryana Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
5716,C10-40 Isoalkyl Acid Cholesterol Esters,NOT RATED,"Functions: Skin Conditioning, Viscosity Contro..."


In [380]:
dataset1_df.describe(include="all")

,ingredient_name,rating,functions
count,26087,26087,26087
unique,26084,6,4056
top,Rubus Idaeus (Raspberry) Fruit Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,2,23675,5642


In [381]:
dataset1_df.ingredient_name.is_unique

False

In [382]:
dataset1_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26082                        Zostera Marina Extract
26083       Zygophyllum Qatarense Leaf/Stem Extract
26084    Zygosaccharomyces Microellipsoides Ferment
26085                     Zymomonas Ferment Extract
26086                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26087, dtype: object>

In [383]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset1_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)


                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26079                          Zymomonas Ferment Extract   
26080                         Zymomonas Ferment Filtrate   
26081                                biosaccharide gum-1   
26082                                        pH Adjuster   
26083                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [384]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset1_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)


                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4056 rows x 1 columns]


#### Explore Tabel dataset2_df

In [385]:
dataset2_df.sample(5)

,ingredient_name,rating,functions
10905,Glycereth-3 Sunflowerate,NOT RATED,"Functions: Cleansing Agent, Surfactant - Foam ..."
25407,Tuber Melanosporum Whole Plant Oil,NOT RATED,Functions: Fragrance. We have not yet rated th...
24331,Tea-Taurine Lauryl Aspartate,NOT RATED,"Functions: Hair Conditioning, Cleansing Agent...."
22600,Silicone Quaternium-27,NOT RATED,Functions: Hair Conditioning. We have not yet ...
16257,Oenothera Tetraptera Leaf Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...


In [386]:
dataset2_df.describe(include="all")

,ingredient_name,rating,functions
count,26074,26074,26074
unique,26074,6,4045
top,Zymomonas Ferment Filtrate,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,1,23675,5642


In [387]:
dataset2_df.ingredient_name.is_unique

True

In [388]:
dataset2_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26069                        Zostera Marina Extract
26070       Zygophyllum Qatarense Leaf/Stem Extract
26071    Zygosaccharomyces Microellipsoides Ferment
26072                     Zymomonas Ferment Extract
26073                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26074, dtype: object>

In [389]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset2_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)

                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26069                          Zymomonas Ferment Extract   
26070                         Zymomonas Ferment Filtrate   
26071                                biosaccharide gum-1   
26072                                        pH Adjuster   
26073                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [390]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset2_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)

                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4045 rows x 1 columns]


##### Menggabungkan dataset1 dan dataset2

In [391]:
datasetAll_df = pd.concat([dataset1_df, dataset2_df], axis=0, ignore_index=True)
datasetAll_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


#### Lowercasing, menghapus karakter khusus, dan tokenisasi


In [392]:
def clean_text(text):
    if isinstance(text, str):
        # Lowercasing
        text = text.lower()
        # Menghapus karakter khusus
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Menghapus spasi berlebih
        text = re.sub(r'\s+', ' ', text).strip()
    return text

In [393]:
datasetAll_df['ingredient_name'] = datasetAll_df['ingredient_name'].apply(clean_text)
datasetAll_df['functions'] =  datasetAll_df['functions'].apply(clean_text)

In [394]:
sentences = []
labels = []
stopwords = [ "a", "an", "in", "is", "the", "and", "also", "there" ]

### Menghapus stopwords dan melakukan lemmatization

In [395]:
def preprocess_text(text):
    # Initialize lemmatizer within the function
    lemmatizer = WordNetLemmatizer()
    if isinstance(text, str):
        # Tokenisasi
        tokens = text.split()
        # Menghapus stopwords dan lemmatization
        processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords]
        return ' '.join(processed_tokens)
    return text

In [396]:
datasetAll_df['functions'] = datasetAll_df['functions'].apply(preprocess_text)

### Export Clean Dataset

In [397]:
# Menyimpan dataset ke file CSV
datasetAll_df.to_csv("Dataset clean/datasetAll.csv", index=False)


In [398]:
dataset1_df.to_csv("Dataset clean/dataset1_clean.csv", index=False)
dataset2_df.to_csv("Dataset clean/dataset2_clean.csv", index=False)

### Pre-processing

In [399]:
# import data
datasetAll_df = pd.read_csv('Dataset clean/datasetAll.csv')
datasetAll_df #show data below

,ingredient_name,rating,functions
0,o ethyl ascorbic acid,BEST,this potent antioxidant highly stable form of ...
1,acai,BEST,pronounced ahsighee this small berry ha deep p...
2,acerola fruit extract,BEST,this fruit extract potent source of antioxidan...
3,acetyl carnitine hcl,BEST,function a antioxidant
4,acetyl dipeptide cetyl ester,BEST,labmade skinfirming peptide formed from reacti...
...,...,...,...
52156,zostera marina extract,NOT RATED,function skin conditioning we have not yet rat...
52157,zygophyllum qatarense leafstem extract,NOT RATED,function antioxidant we have not yet rated thi...
52158,zygosaccharomyces microellipsoides ferment,NOT RATED,function humectant we have not yet rated this ...
52159,zymomonas ferment extract,NOT RATED,function humectant skin conditioning we have n...


In [400]:
print(datasetAll_df['rating'].unique())

['BEST' 'GOOD' 'AVERAGE' 'BAD' 'WORST' 'NOT RATED']


In [401]:
# Mengonversi nilai non-numerik ke angka
rating_mapping = {
    'BEST': 6,
    'GOOD': 5,
    'AVERAGE': 4,
    'BAD': 3,
    'WORST':2,
    'NOT RATED':1
}

datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)

C:\Users\Windows\AppData\Local\Temp\ipykernel_6168\567315464.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)


In [402]:
# Hapus baris yang tidak sesuai dengan konteks
datasetAll_df = datasetAll_df[datasetAll_df['rating'].apply(lambda x: str(x).isnumeric() or x in rating_mapping.keys())]


In [403]:
datasetAll_df['rating'] = pd.to_numeric(datasetAll_df['rating'], errors='coerce')

In [404]:
# Encoding categorical data (jika ada kolom kategori)
from sklearn.preprocessing import LabelEncoder

# Misal kolom 'functions' merupakan data kategori
le = LabelEncoder()
datasetAll_df['functions'] = le.fit_transform(datasetAll_df['functions'])

# Normalisasi atau Standardisasi data numerik
scaler = StandardScaler()

# Misalkan kolom numerik adalah ['rating'] (pastikan kolom ini ada)
datasetAll_df[['rating']] = scaler.fit_transform(datasetAll_df[['rating']])

# Split dataset menjadi train dan test
X = datasetAll_df.drop(columns=['ingredient_name', 'functions'])  # Fitur (tanpa target)
y = datasetAll_df['functions']  # Target (misalnya 'functions')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Periksa ukuran data
print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

Train size: (41728, 1)
Test size: (10433, 1)


In [405]:
datasetAll_df

,ingredient_name,rating,functions
0,o ethyl ascorbic acid,3.937776,3778
1,acai,3.937776,3236
2,acerola fruit extract,3.937776,3665
3,acetyl carnitine hcl,3.937776,631
4,acetyl dipeptide cetyl ester,3.937776,2731
...,...,...,...
52156,zostera marina extract,-0.302609,2463
52157,zygophyllum qatarense leafstem extract,-0.302609,1205
52158,zygosaccharomyces microellipsoides ferment,-0.302609,2301
52159,zymomonas ferment extract,-0.302609,2293


In [406]:
X_train.to_csv("Dataset clean/X_train.csv", index=False)
X_test.to_csv("Dataset clean/X_test.csv", index=False)
y_train.to_csv("Dataset clean/y_train.csv", index=False)
y_test.to_csv("Dataset clean/y_test.csv", index=False)

### Prepare the dataset for training

In [407]:
vocab_size = 15000
embedding_dim = 32
max_length = 200
trunc_type='post'
padding_type='post'
oov_token = "<OOV>"
training_portion = 0.8

In [408]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert labels to numpy arrays
train_labels = np.array(y_train)
test_labels = np.array(y_test)

In [409]:
# Mengecek jumlah sequence sebelum padding
print("Jumlah sequence dalam train_sequences:", len(train_sequences))
print("Jumlah sequence dalam test_sequences:", len(test_sequences))

# Mengecek panjang setiap sequence sebelum padding
train_lengths = [len(seq) for seq in train_sequences]
test_lengths = [len(seq) for seq in test_sequences]

# Menampilkan beberapa panjang sequence untuk memverifikasi
print("Panjang beberapa sequence dalam train_sequences:", train_lengths[:10])  # Menampilkan 10 sequence pertama
print("Panjang beberapa sequence dalam test_sequences:", test_lengths[:10])  # Menampilkan 10 sequence pertama

# Menampilkan distribusi panjang sequence untuk melihat seberapa bervariasi panjangnya
from collections import Counter
train_length_distribution = Counter(train_lengths)
test_length_distribution = Counter(test_lengths)

print("Distribusi panjang sequence dalam train_sequences:", train_length_distribution)
print("Distribusi panjang sequence dalam test_sequences:", test_length_distribution)


Jumlah sequence dalam train_sequences: 1
Jumlah sequence dalam test_sequences: 1
Panjang beberapa sequence dalam train_sequences: [1]
Panjang beberapa sequence dalam test_sequences: [1]
Distribusi panjang sequence dalam train_sequences: Counter({1: 1})
Distribusi panjang sequence dalam test_sequences: Counter({1: 1})


In [410]:
with open("Dataset clean/datasetAll.csv", 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

In [411]:
file_path = 'Dataset clean/datasetAll.csv'


In [412]:
# Menghitung ukuran training dan validation set
train_size = int(len(sentences) * training_portion)
validation_size = int(len(sentences) * 0.1)  # Misalnya 10% untuk validasi

# Membagi dataset
train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:train_size + validation_size]
validation_labels = labels[train_size:train_size + validation_size]

test_sentences = sentences[train_size + validation_size:]
test_labels = labels[train_size + validation_size:]

In [413]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)



In [414]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

# Convert the labels to sequences
training_label_seq = label_tokenizer.texts_to_sequences(train_labels)
validation_label_seq = label_tokenizer.texts_to_sequences(validation_labels)

# Pad the sequences to ensure they have the same length
training_label_seq = pad_sequences(training_label_seq, padding='post')
validation_label_seq = pad_sequences(validation_label_seq, padding='post')

# Check the shapes of padded sequences
print(f"Train label sequence shape: {training_label_seq.shape}")
print(f"Validation label sequence shape: {validation_label_seq.shape}")


Train label sequence shape: (41728, 14)
Validation label sequence shape: (5216, 10)


In [415]:
print(training_label_seq.dtype)
print(validation_label_seq.dtype)

int32
int32


# Mengecek Padding sesuai atau tidak dengan max_length

In [416]:
# Mengecek jumlah data sebelum dan sesudah pemisahan
print("Jumlah data sebelum split:", len(datasetAll_df['ingredient_name']))
print("Jumlah data dalam train_sentences:", len(train_sentences))
print("Jumlah data dalam test_sentences:", len(test_sentences))

# Tokenizer dan padding
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

# Cek panjang sequence sebelum dan sesudah padding
print(f"Panjang sequence pertama sebelum padding: {len(train_sequences[0])}")

# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Cek panjang sequence setelah padding
print(f"Panjang sequence pertama setelah padding: {len(train_padded[0])}")

Jumlah data sebelum split: 52161
Jumlah data dalam train_sentences: 41728
Jumlah data dalam test_sentences: 5217
Panjang sequence pertama sebelum padding: 1
Panjang sequence pertama setelah padding: 200


In [418]:
# Tampilkan panjang setiap sequence
print("Panjang setiap sequence:", [len(seq) for seq in train_padded])

# Cek isi sequence pertama
print("Contoh sequence pertama:", train_padded[0])

# Cek jika ada lebih dari satu sequence
if len(train_padded) > 1:
    print("Contoh sequence kedua:", train_padded[1])
else:
    print("Data hanya memiliki satu sequence.")

Panjang setiap sequence: [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,

In [419]:
# Mengecek jumlah data sebelum dan sesudah pemisahan
print("Jumlah data sebelum split:", len(datasetAll_df['ingredient_name']))
print("Jumlah data dalam train_sentences:", len(train_sentences))
print("Jumlah data dalam test_sentences:", len(test_sentences))

Jumlah data sebelum split: 52161
Jumlah data dalam train_sentences: 41728
Jumlah data dalam test_sentences: 5217
